In [1]:
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
import re
import pathlib
import numpy as np
import pickle
from IPython.display import display
from docx import Document
from pprint import pprint
from collections import defaultdict
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [17]:
def 生成語料描述統計pickle檔(corpus_pickel_path, output_date):
    
    df = pd.read_pickle(corpus_pickel_path, compression="gzip")
    
    # 先算各篇文本的總和(先不拆分子文本)  
    df['總詞數 (族語)'] = df['Ab'].str.split(' ').str.len()
    df['總詞數 (族語)'] = df['總詞數 (族語)'].fillna(0)
    df['總詞數 (族語)'] = df['總詞數 (族語)'].astype(int)
    df['總句數 (族語)'] = df['Ab'].astype('str').apply(lambda s: len(sent_tokenize(s)))
    df['總句數 (族語)'] = df['總句數 (族語)'].astype('int')
    df_each_text = df.groupby(['From']).sum().reset_index()
    df_each_text.rename(columns = {'From':'文本來源'}, inplace = True)
    display(df_each_text)
    df_each_text.to_pickle(f'./data/stats_of_each_text_{output_date}.pkl', compression="gzip")
    
    # 再處理含子文本的統計
    df_each_text_with_subtext = df[~df['Source'].isna()]
    df_each_text_with_subtext['article_id'] = df_each_text_with_subtext.Source.str.replace('_\d+', '')
    df_each_text_with_subtext = df_each_text_with_subtext.groupby(['From', 'article_id']).sum().reset_index()
    df_each_text_with_subtext.rename(columns = {'From':'文本來源', 'article_id': '子文本篇目'}, inplace = True)
    display(df_each_text_with_subtext)
    df_each_text.to_pickle(f'./data/stats_of_each_text_with_subtext_{output_date}.pkl', compression="gzip")
    
生成語料描述統計pickle檔('./data/Formosan-Mandarin_sent_pairs_20221227-2.pkl', '20221228')

,文本來源,總詞數 (族語),總句數 (族語)
0,Egerod (1974),996,186
1,Huang & Wu (2016),1286,86
2,Huang (1993),1954,183
3,Huang (1994),747,57
4,Rau (1992),3659,234
5,Rau et al. (1995),811,53
6,九階教材,1620,232
7,傳說故事精選篇,8789,840
8,句型,6984,1436
9,大嵙崁群故事,7265,362


C:\Users\Owner\AppData\Local\Temp\ipykernel_9328\3992975153.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df_each_text_with_subtext['article_id'] = df_each_text_with_subtext.Source.str.replace('_\d+', '')
C:\Users\Owner\AppData\Local\Temp\ipykernel_9328\3992975153.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_each_text_with_subtext['article_id'] = df_each_text_with_subtext.Source.str.replace('_\d+', '')


,文本來源,子文本篇目,總詞數 (族語),總句數 (族語)
0,Huang & Wu (2016),1,642,43
1,Huang & Wu (2016),2,644,43
2,Rau (1992),1,860,56
3,Rau (1992),2,828,49
4,Rau (1992),4,1406,75
5,Rau (1992),6-1,145,14
6,Rau (1992),6-2,100,8
7,Rau (1992),6-3,159,17
8,Rau (1992),6-4,65,7
9,Rau (1992),6-5,51,4
